In [211]:
import CSV
using DataFrames
# using StatsBase
include("pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
import Statistics

In [2]:
using Turing, Distributions

using MCMCChains, StatsPlots

using StatsFuns: logistic

using Random
Random.seed!(0);

using StatsBase
import MLJBase

In [3]:
MissingImputator = @load MissingImputator pkg="BetaML" verbosity=0
XGBoostClassifier = @load XGBoostClassifier verbosity=0  # loads code defining a model type
LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0;  # loads code defining a model type
AdaBoostStumpClassifier = @load AdaBoostStumpClassifier verbosity=0;  # loads code defining a model type
EvoTreeClassifier = @load EvoTreeClassifier verbosity=0
GradientBoostingClassifier = @load GradientBoostingClassifier verbosity=0
EvoTreeGaussian = @load EvoTreeGaussian verbosity=0
SVC = @load SVC verbosity=0

MLJLIBSVMInterface.SVC

In [575]:
sample_key, xold, yold = FileIO.load("cleaned_data.jld2","sample_key","x" ,"y");
xold.col1=collect(1:nrow(xold));
filter!(x->!ismissing(x.gt_AF_Mutect),xold);

In [576]:
for j in 1:ncol(xold), i in 1:nrow(xold)
    if !ismissing(xold[i,j]) && (isnan(xold[i,j]) || isinf(xold[i,j]) )
        xold[i,j] = missing
    end
end

In [577]:
coldrop = []
for i in names(xold)
    if i == "col1"
        continue
    end
    if length(unique(xold[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Vardict",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop);

In [578]:
select!(xold, Not(coldrop));
yold = yold[xold.col1]
sample_key=sample_key[xold.col1]
select!(xold,Not("col1"));
println(size(xold))
println(size(yold))
println(size(sample_key))

(47003, 83)
(47003,)
(47003,)


In [586]:
xold.sample_key=sample_key
xold.real=[i==true for i in yold]
filter(x->x.training>0.5 && x.real && (ismissing(x.MPOS_Mutect) || x.MPOS_Mutect < 30),xold)[:,["MPOS_Mutect","sample_key","real"]]

,MPOS_Mutect,sample_key,real
,Float32?,String,Bool
1,25.0,218281_1_100 2:25457243 G>A,1
2,25.0,218281_1_10 2:25457243 G>A,1
3,25.0,218281_T 2:25457243 G>A,1
4,missing,272341_1_1000 4:106155751 G>A,1
5,missing,272341_1_1000 4:106158216 G>A,1
6,missing,272341_1_1000 4:106190862 T>C,1
7,28.0,736399_1_100 4:106190822 C>A,1
8,28.0,736399_1_100 4:106196829 T>G,1
9,29.0,736399_1_100 4:106196951 A>G,1


In [557]:
# namd=names(xsnv)[13] # 2,6,8,9,13
# namd="MBQ_Mutect_1"
namd="MPOS_Mutect"
xold.sample_key=sample_key
xold.real=[i==true for i in yold]
print(namd)
histogram(collect(skipmissing(filter(x->x.training>0.5 && x.real,xold)[:,namd])),legend=true,label="TT",alpha=0.3,bins=10)
histogram!(collect(skipmissing(filter(x->x.training<0.5 && x.real,xold)[:,namd])),label="BMT",alpha=0.3,bins=10)



LoadError: ArgumentError: New columns must have the same length as old columns

In [480]:
xsnv = copy(xold);
ysnv = copy(yold);


# train2 = findall(x->x,[i>0.5 for i in xsnv.training])
# test = findall(x->x,[i<0.5 for i in xsnv.training])


train2, test = partition(eachindex(ysnv), 0.7, stratify=ysnv,  shuffle=true, rng=12344);

select!(xsnv,Not("training"));

In [481]:
transform_standardizer!(xsnv,fit_standardizer(xsnv[train2,:]));

In [482]:
# showtable(xsnv[1:10,:])

In [483]:
sum(ismissing.(Matrix(xsnv)))

4137

In [484]:
stmach = machine(MissingImputator(K=5),xsnv[train2,:]) 
@time sttran = fit!(stmach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…} @888.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/W4gFl/src/machines.jl:391


 12.455284 seconds (68.28 M allocations: 12.513 GiB, 17.42% gc time)


Machine{MissingImputator,…} @888 trained 1 time; caches data
  args: 
    1:	Source @970 ⏎ `Table{Union{AbstractVector{ScientificTypesBase.Continuous}, AbstractVector{Union{Missing, ScientificTypesBase.Continuous}}}}`


In [485]:
tt = MLJ.transform(sttran,xsnv);
for i in 1:ncol(xsnv)
    xsnv[:,i] = tt[i]
end

In [486]:
disallowmissing!(xsnv);

In [519]:
train = copy(train2)
x = copy(xsnv);
y = copy(ysnv);

In [520]:
x,y,train = sampler(x,y,train,"over",0.5);

ratio: 0.0026512265732134693
number of new: 16320


In [521]:
y = coerce(copy(y),OrderedFactor);

In [522]:
println(size(x[train,:])," ",sum([i==true for i in y[train]] ) )
println(size(x[test,:])," ",sum([i==true for i in y[test]] ) )
println(size(y[train]))

(49222, 82) 16407
(14101, 82) 37
(49222,)


# MLJ Models

In [523]:

r = [
    range(EvoTreeClassifier(), :max_depth, lower=1,upper=5),
    range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
    range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
    
    
#     range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
#     range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
    
#     range(SVC(), :degree, lower=1,upper=4),
#     range(SVC(), :gamma, lower=0.0,upper=1.0),    
]

7-element Vector{MLJBase.NumericRange{T, MLJBase.Bounded, Symbol} where T}:
 typename(MLJBase.NumericRange)(Int64, :max_depth, ... )
 typename(MLJBase.NumericRange)(Float64, :η, ... )
 typename(MLJBase.NumericRange)(Float64, :γ, ... )
 typename(MLJBase.NumericRange)(Float64, :λ, ... )
 typename(MLJBase.NumericRange)(Float64, :α, ... )
 typename(MLJBase.NumericRange)(Float64, :min_weight, ... )
 typename(MLJBase.NumericRange)(Int64, :nrounds, ... )

In [540]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
#     model=SVC(),
    model=EvoTreeClassifier(),
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=20,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
#     measure = brier_loss # use for probabalistic 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x[train,:],y[train])
# fit!(machbest,verbosity=1 )
evaluate!(machbest,verbosity=2,resampling=StratifiedCV(nfolds=5, rng=123) )


┌ Info: Training Machine{XGBoostClassifier,…} @176.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/W4gFl/src/machines.jl:391
[1]	train-logloss:0.445040
[2]	train-logloss:0.307910
[3]	train-logloss:0.221587
[4]	train-logloss:0.161240
[5]	train-logloss:0.119526
[6]	train-logloss:0.089146
[7]	train-logloss:0.067344
[8]	train-logloss:0.052089
[9]	train-logloss:0.040240
[10]	train-logloss:0.031409
[11]	train-logloss:0.025139
[12]	train-logloss:0.019271
[13]	train-logloss:0.014970
[14]	train-logloss:0.011890
[15]	train-logloss:0.009565
[16]	train-logloss:0.007547
[17]	train-logloss:0.006046
[18]	train-logloss:0.005035
[19]	train-logloss:0.004328
[20]	train-logloss:0.003747
[21]	train-logloss:0.003131
[22]	train-logloss:0.002681
[23]	train-logloss:0.002364
[24]	train-logloss:0.001931
[25]	train-logloss:0.001707
[26]	train-logloss:0.001537
[27]	train-logloss:0.001359
[28]	train-logloss:0.001235
[29]	train-logloss:0.001065
[30]	train-logloss:0.000929
[31]	train-logloss:0.000822
[32]	tra

[78]	train-logloss:0.000165
[79]	train-logloss:0.000163
[80]	train-logloss:0.000162
[81]	train-logloss:0.000159
[82]	train-logloss:0.000158
[83]	train-logloss:0.000157
[84]	train-logloss:0.000156
[85]	train-logloss:0.000155
[86]	train-logloss:0.000153
[87]	train-logloss:0.000152
[88]	train-logloss:0.000151
[89]	train-logloss:0.000150
[90]	train-logloss:0.000149
[91]	train-logloss:0.000147
[92]	train-logloss:0.000146
[93]	train-logloss:0.000145
[94]	train-logloss:0.000144
[95]	train-logloss:0.000143
[96]	train-logloss:0.000142
[97]	train-logloss:0.000141
[98]	train-logloss:0.000140
[99]	train-logloss:0.000140
[100]	train-logloss:0.000139
Evaluating over 5 folds:  60%[===============>         ]  ETA: 0:00:32┌ Info: Training Machine{XGBoostClassifier,…} @176.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/W4gFl/src/machines.jl:391
[1]	train-logloss:0.442856
[2]	train-logloss:0.304754
[3]	train-logloss:0.217836
[4]	train-logloss:0.157214
[5]	train-logloss:0.115781
[6]	train-logloss

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────────────────────────────┬─────────────┬───────────┬───────
│ measure                                     │ measurement │ operation │ per_ ⋯
├─────────────────────────────────────────────┼─────────────┼───────────┼───────
│ LogLoss(tol = 2.22045e-16)\e @674\e │ 0.000692    │ predict   │ Floa ⋯
└─────────────────────────────────────────────┴─────────────┴───────────┴───────
                                                                1 column omitted


In [541]:
yhat=MLJ.predict_mode(machbest, x[test,:]); # probab
# yhat=MLJ.predict(machbest, x[test,:]); # determ

In [542]:
ConfusionMatrix()(yhat,y[test])

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │    false    │    true     │
├─────────────┼─────────────┼─────────────┤
│    false    │    14056    │     10      │
├─────────────┼─────────────┼─────────────┤
│    true     │      8      │     27      │
└─────────────┴─────────────┴─────────────┘


In [529]:
report(machbest).best_report.feature_importances # Evo Tree
# fitted_params(machbest).best_fitted_params.coefs[sortperm([i[2] for i in fitted_params(machbest).best_fitted_params.coefs])]



82-element Vector{Pair{String, Float64}}:
               "FILTER_Mutect_strand_bias" => 0.5815291608476004
                           "MFRL_Mutect_2" => 0.14514253582769632
 "total_greater_than_min_alt_count_Mutect" => 0.0984383130301204
                            "total_Mutect" => 0.05161511109760357
                             "MPOS_Mutect" => 0.024147254701723412
                              "ROQ_Mutect" => 0.022817746816445446
                       "dust_score_Mutect" => 0.017364040181786555
                            "MBQ_Mutect_2" => 0.01523401621201679
                                 "alt_len" => 0.015142995422258762
                 "VARIANT_CLASS_insertion" => 0.013245623957398877
                           "RefRev_Mutect" => 0.005660876175123507
                            "MBQ_Mutect_1" => 0.004939894655197984
                        "gt_AD_alt_Mutect" => 0.0027025416112541126
                                           ⋮
                                   "REF_I" => 0.

In [530]:
aa = DataFrame(x1=yhat,x2=y[test],sample_key=sample_key[test])
filter(x-> x.x1==false && x.x2==true ,aa).sample_key

6-element Vector{String}:
 "D-953 2:25469600 T>TG"
 "D-1002 X:39921513 C>T"
 "D-1002 X:39922877 G>T"
 "D-1002 X:39932574 G>T"
 "736399_1_100 4:106196829 T>G"
 "D-810 X:39914759 G>A"

In [531]:
MLJ.predict(machbest, x[test,:])

14101-element MLJBase.UnivariateFiniteVector{Multiclass{2}, Bool, UInt8, Float64}:
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.399, true=>0.601)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 ⋮
 UnivariateFinite{Multiclass{2}}(false=>0.646, true=>0.354)
 UnivariateFinite{Multiclass{2}}(false=>0.612, true=>0.388)
 UnivariateFin